In [1]:
#Install the nessasary packages from pip
!pip install mediapipe opencv-python pandas scikit-learn --user

In [83]:
#Import all the nessacary libraries from the packages

#Import mediapipe to be use as the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import csv #Handle csv files
import os #Handle folder structure
import glob
import pickle #Save and oad ML model

from sklearn.model_selection import train_test_split #Partition the data into training and testing

from sklearn.pipeline import make_pipeline #Creates a pipeline
from sklearn.preprocessing import StandardScaler #Standadize data 

#Classification algorithm
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score #Evaluate model through accuracy

In [2]:
#Helper to draw the landmarks and provide the landmark detection models
draw_helpers = mp.solutions.drawing_utils 
holistic_model = mp.solutions.holistic 

In [4]:
#DETECT AND SAVE THE LANDMARKS COORDS AS COLUMNS FROM THE SAMPLE VIDEO

#Connect the sample video from the device
sample_video = cv2.VideoCapture('1.valid.mp4')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Exit!")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
        draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
         
        #Display the frames    
        cv2.imshow('Raw Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
sample_video.release()
cv2.destroyAllWindows()

Cannot receive frame!


In [5]:
#Store the pose landmarks in an array 
landmarks_array = result_frame.pose_landmarks.landmark

#Save the no of pose landmarks in the Array 
num_landmarks = len(landmarks_array)
#Display landmarks
num_landmarks

33

In [6]:
#Save the landmarks to a table to be exported as a csv file

#0th column of the table
table_columns = ['class']
#Add columns to the table according to the no.of landmarks
for num in range(1, num_landmarks + 1):
    table_columns += ['x{}'.format(num), 'y{}'.format(num), 'z{}'.format(num), 'v{}'.format(num)]
    
#Display columns of the table
table_columns

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [7]:
#Write to the csv file
with open('data.csv', mode='w', newline='') as f:
    #Define the csv writer
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(table_columns)

In [81]:
#DETECT AND SAVE COORDS OF LANDMARKS FOR EACH CLASS THROUGH THE TRAIN VIDEO

def video_processor(class_name, video_pth):
    #Connect the sample video from the device
    sample_video = cv2.VideoCapture(video_pth)

    #Load the holistic model
    with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        #Loop through each frame of the video 
        while sample_video.isOpened():
            #Returns the status of the read and the frame as an image
            status, frame = sample_video.read()

            #If frame is read correctly, status is true
            if status == False:
                print("Exit!")
                break

            #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            #Prevent writing and copying frame data to improve performance while making the detection
            rgb_frame.flags.writeable = False        

            #Use holistic model to make detections
            result_frame = holistic.process(rgb_frame)

            #Set frame back to writable format after detection
            rgb_frame.flags.writeable = True   

            #Recolor the captured frame from BGR for rendering with opencv
            bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

            #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
            draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                     draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )
         
            #Export the coordinates of the landmarks to the csv file
            try:

                #Extracting all the landmarks of the pose as an array
                pose_landmarks_array = result_frame.pose_landmarks.landmark
                #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
                pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten() 
                                              if result_frame.pose_landmarks else np.zeros(33*4))

                #Append class name as the Oth element
                pose_landmarks_nparray.insert(0, class_name)
                
                #Append the data to table in the csv file
                with open('data.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(pose_landmarks_nparray) 
            except:
                pass
            
            #Display the frames    
            cv2.imshow('Raw Feed', bgr_frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    sample_video.release()
    cv2.destroyAllWindows()

In [82]:
video_processor("Correct", "3.valid.mp4")

Exit!


In [120]:
#Specify the location to the datasets
valid_path = "./datasets/valid/*.mp4"
invalid_path = "./datasets/invalid/*.mp4"

#Displat the no of files in the datasets
print("Valid Video Count: ", len(glob.glob(valid_path)))
print("Invalid Video Count: ", len(glob.glob(invalid_path)))

Valid video count :  30
Invalid video count : 11


In [ ]:
#Adding landmarks of the invalid dataset to the csv
class_name = "Correct"
for i in range (1, len(glob.glob(valid_path)) + 1):
    video_pth = "./datasets/valid/" + str(i) + ".mp4"
    video_processor(class_name, video_pth)
    print("Processed: ", str(i)

In [ ]:
#Adding landmarks of the invalid dataset to the csv
class_name = "Incorrect"
for i in range (1, len(glob.glob(invalid_path)) + 1):
    video_pth = "./datasets/invalid/" + str(i) + ".mp4"
    video_processor(class_name, video_pth)
    print("Processed: ", str(i)

In [26]:
#Import dataframe
df = pd.read_csv('data.csv')

In [27]:
#Display the first 5 rows in the dataframe
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Correct,0.534789,0.340018,-0.734422,0.999999,0.545061,0.327873,-0.688252,0.999999,0.552311,...,-0.472349,0.937949,0.645837,0.759634,-0.685680,0.975963,0.406702,0.761222,-0.760625,0.981579
1,Correct,0.534788,0.340075,-0.679886,0.999999,0.544802,0.327912,-0.635636,0.999999,0.552053,...,-0.587094,0.938885,0.649047,0.759775,-0.824108,0.976163,0.405543,0.761442,-0.881457,0.981426
2,Correct,0.534851,0.340163,-0.643253,0.999999,0.544784,0.328017,-0.599718,0.999999,0.552030,...,-0.594323,0.939786,0.650265,0.759939,-0.822347,0.976234,0.404899,0.761539,-0.884187,0.981182
3,Correct,0.534901,0.340234,-0.639145,0.999999,0.544768,0.328118,-0.595136,0.999999,0.552000,...,-0.601119,0.940453,0.650691,0.760146,-0.839792,0.976385,0.404781,0.761661,-0.892310,0.980968
4,Correct,0.535042,0.340296,-0.648428,0.999999,0.544791,0.328195,-0.604258,0.999999,0.552007,...,-0.596905,0.940832,0.650841,0.760248,-0.832493,0.976366,0.404780,0.761688,-0.888286,0.980666


In [28]:
#Display the last 5 rows in the dataframe
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
76,Correct,0.527193,0.341830,-0.696658,0.999995,0.535855,0.329449,-0.653082,0.999994,0.541785,...,-0.477941,0.931223,0.649873,0.760314,-0.699613,0.974437,0.405546,0.762178,-0.757666,0.977532
77,Correct,0.527273,0.341830,-0.691384,0.999996,0.535877,0.329460,-0.647774,0.999994,0.541833,...,-0.490456,0.931123,0.649910,0.760346,-0.720183,0.974255,0.405584,0.762178,-0.770985,0.977179
78,Correct,0.527475,0.341830,-0.682671,0.999996,0.535949,0.329476,-0.640857,0.999994,0.541972,...,-0.505001,0.930721,0.650216,0.760388,-0.744007,0.974045,0.405910,0.762099,-0.785265,0.976663
79,Correct,0.527481,0.341830,-0.668001,0.999996,0.535951,0.329490,-0.627740,0.999995,0.541982,...,-0.495164,0.930214,0.650356,0.760399,-0.727630,0.973676,0.406083,0.762012,-0.772083,0.976117
80,Correct,0.527493,0.341825,-0.668258,0.999996,0.535957,0.329493,-0.627790,0.999995,0.542005,...,-0.492280,0.929735,0.650851,0.760377,-0.719618,0.973319,0.406098,0.761930,-0.766681,0.975687


In [30]:
#Remove the class column so the dataframe only contains features
X = df.drop('class', axis=1)
#Use the class as the target value
Y = df['class'] 

0     Correct
1     Correct
2     Correct
3     Correct
4     Correct
       ...   
76    Correct
77    Correct
78    Correct
79    Correct
80    Correct
Name: class, Length: 81, dtype: object

In [31]:
#Split the data with 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

In [84]:
#Setup the machine learning model pipeline
model_pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

In [85]:
#Train the model
model = model_pipeline.fit(X_train.values, Y_train.values)

In [86]:
#Predict the class of the test data
output_class = model.predict(X_test.values)

#Comparing the output labels with the test data labels
print("Accuracy of the model: ", accuracy_score(Y_test, output_class))

Accuracy of the model:  1.0


In [35]:
#Save the model as a binary file
with open('rfc_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [36]:
#Import the model from the binary file
with open('rfc_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [58]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [80]:
#PREDICT AND DISPLAY THE RESULTS OF THE MODEL BY PASSING THE TEST VIDEO

#Connect the test video from the device
sample_video = cv2.VideoCapture('5.invalid.mp4')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Exit!")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
        draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
         #Predict the coordinates of the landmarks (resulrs screen)
        try:
            #Extracting all the landmarks of the pose as an array
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten() 
                                          if result_frame.pose_landmarks else np.zeros(33*4))
            
            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])

            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]
            
            
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            #Display the class label inside the rectangle box
            cv2.putText(bgr_frame, 'Class'
                        , (105,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract =and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (100,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Probability'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass
                        
        #Display the frames  

        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    print("Class:", pose_class_status, " Probability:", str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2)))


sample_video.release()
cv2.destroyAllWindows()

Exit!
Class: Correct  Probability: 1.0
